# Enhanced Fabric Workspace Scanner v02

## Features Added from Version 1:
- **Lakehouse Storage**: Saves all analysis results to dedicated lakehouse tables
- **Enhanced Context**: Additional context columns for Reports, Tables, Relationships, Dataflows
- **Column Usage Analysis**: Detailed column usage analysis with context from measures, relationships, and dependencies
- **Spark Integration**: Uses Spark DataFrames for efficient storage

## Tables Created in Lakehouse:
- `workspace_analysis` - Workspace information
- `dataset_analysis` - Datasets with Reports, Tables, Relationships, Dataflows context
- `table_analysis` - Tables with usage context from measures, relationships, dependencies
- `column_usage_analysis` - Columns with detailed usage analysis
- `usage_summary` - Summary of dataset usage patterns


In [ ]:
# Install semantic-link-labs for extended Fabric analytics
!pip install semantic-link-labs

In [ ]:
import pandas as pd
import sempy_labs
import sempy.fabric as fabric
from sempy_labs.report import ReportWrapper
import re
import sempy
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.types import ArrayType, StringType, StructType, LongType, StructField, FloatType
from pyspark.sql.functions import col
from datetime import datetime
import time

# Initialize Spark session
spark = SparkSession.builder.getOrCreate()

print("✅ All imports successful and Spark session initialized")

In [ ]:
def sanitize_df_columns(df, extra_columns=False, ws_id=None, ds_id=None):
    """
    Replaces spaces in column names with underscore to prevent errors during Spark Dataframe Creation
    """
    if df.empty:
        return df
        
    df.columns = [
        re.sub(r'\W+', "_", col.strip().lower())
        for col in df.columns
    ]

    if extra_columns:
        df['workspace_id'] = ws_id
        df['dataset_id'] = ds_id
        
    return df

def save_to_lakehouse(df, table_name, description=""):
    """
    Save DataFrame to lakehouse using Spark - extracted from Version 1
    """
    try:
        if df.empty:
            print(f"  ⚠️ Skipping empty DataFrame for table: {table_name}")
            return
            
        # Add analysis timestamp
        df_with_timestamp = df.copy()
        df_with_timestamp['analysis_date'] = datetime.now()
        
        # Convert to Spark DataFrame and save
        spark_df = spark.createDataFrame(df_with_timestamp)
        spark_df.write.mode("overwrite").saveAsTable(table_name)
        
        print(f"  ✅ Saved {len(df)} records to '{table_name}' table")
        if description:
            print(f"     📝 {description}")
            
    except Exception as e:
        print(f"  ❌ Error saving to {table_name}: {str(e)}")

print("✅ Utility functions defined")

In [ ]:
# ------------------------------------------------------------
# STEP 1: Object Discovery
# ------------------------------------------------------------

print("🔍 Discovering workspaces...")

workspaces_df = fabric.list_workspaces()
workspaces_df = sanitize_df_columns(workspaces_df)
workspaces_df = workspaces_df[['id', 'name', 'type']]
display(workspaces_df)

datasets_all, reports_all, paginated_all, dataflows_all = [], [], [], []

for _, ws in workspaces_df.iterrows():
    ws_id = ws['id']
    ws_name = ws['name']
    ws_type = ws['type']
    if ws_type == "AdminInsights":
        continue
    print(f"\n📦 Scanning workspace: {ws_name}")

   # --- Datasets
    try:
        ds = fabric.list_datasets(workspace=ws_id)
        if not ds.empty:
            ds['workspace_id'] = ws_id
            ds['workspace_name'] = ws_name
            datasets_all.append(ds)
    except Exception as e:
        print(f"  ⚠️ Datasets error in {ws_name}: {e}")

    # --- Reports (includes both Power BI and Paginated)
    try:
        rep = fabric.list_reports(workspace=ws_id)
        if not rep.empty:
            rep['workspace_id'] = ws_id
            rep['workspace_name'] = ws_name
            reports_all.append(rep)
    except Exception as e:
        print(f"  ⚠️ Reports error in {ws_name}: {e}")

    # --- Dataflows
    try:
        dfs = fabric.list_items(type='Dataflow',workspace=ws_id)
        if not dfs.empty:
            dfs['workspace_id'] = ws_id
            dfs['workspace_name'] = ws_name
            dataflows_all.append(dfs)
    except Exception as e:
        print(f"  ⚠️ Dataflows error in {ws_name}: {e}")

# Combine results
datasets_df  = sanitize_df_columns(pd.concat(datasets_all, ignore_index=True) if datasets_all else pd.DataFrame())
reports_df   = sanitize_df_columns(pd.concat(reports_all, ignore_index=True) if reports_all else pd.DataFrame())
dataflows_df = sanitize_df_columns(pd.concat(dataflows_all, ignore_index=True) if dataflows_all else pd.DataFrame())

# Split report types for clarity
if not reports_df.empty and "report_type" in reports_df.columns:
    pbi_reports_df = reports_df[reports_df["report_type"] == "PowerBIReport"].copy()
    paginated_reports_df = reports_df[reports_df["report_type"] == "PaginatedReport"].copy()
else:
    pbi_reports_df = reports_df
    paginated_reports_df = pd.DataFrame()

print("\n✅ Object discovery complete.")
print(f"  Workspaces: {len(workspaces_df)}")
print(f"  Datasets:   {len(datasets_df)}")
print(f"  Reports:    {len(reports_df)}")
print(f"  Paginated:  {len(paginated_reports_df)}")
print(f"  Dataflows:  {len(dataflows_df)}")

# 🆕 Save to Lakehouse - Workspaces
print("\n💾 Saving workspace data to lakehouse...")
save_to_lakehouse(workspaces_df, "workspace_analysis", "Basic workspace information")

In [ ]:
# ------------------------------------------------------------
# STEP 2: Usage Analysis
# ------------------------------------------------------------

print("\n🔎 Analyzing dataset usage...")

# 1️⃣ Dataset IDs used by any report (Power BI or Paginated)
used_dataset_ids = set()
if not reports_df.empty:
    used_dataset_ids.update(reports_df['dataset_id'].dropna().unique())

# 2️⃣ Dataset IDs used by dataflows (as sources)
dataflow_refs = []

for _, row in dataflows_df.iterrows():
    try:
        refs = sempy_labs.get_dataflow_references(row['id'], row['workspace_id'])
        if refs is not None and not refs.empty:
            refs['dataflow_id'] = row['id']
            refs['dataflow_name'] = row['name']
            refs['workspace_id'] = row['workspace_id']
            dataflow_refs.append(refs)
    except Exception:
        pass

dataflow_refs_df = pd.concat(dataflow_refs, ignore_index=True) if dataflow_refs else pd.DataFrame()

if not dataflow_refs_df.empty:
    if 'source_dataset_id' in dataflow_refs_df.columns:
        used_dataset_ids.update(dataflow_refs_df['source_dataset_id'].dropna().unique())

# 3️⃣ Determine unused datasets
unused_datasets_df = datasets_df[~datasets_df['dataset_id'].isin(used_dataset_ids)].copy()

print(f"✅ Found {len(unused_datasets_df)} potentially unused datasets.")

# 🆕 Enhanced Dataset Analysis with Context from Version 1
print("\n📊 Creating enhanced dataset analysis with context...")

# Add context columns for each dataset
enhanced_datasets = datasets_df.copy()
if not enhanced_datasets.empty:
    enhanced_datasets['report_count'] = 0
    enhanced_datasets['dataflow_count'] = 0
    enhanced_datasets['table_count'] = 0
    enhanced_datasets['relationship_count'] = 0
    enhanced_datasets['is_used'] = enhanced_datasets['dataset_id'].isin(used_dataset_ids)
    
    # Count reports per dataset
    if not reports_df.empty:
        report_counts = reports_df.groupby('dataset_id').size().to_dict()
        enhanced_datasets['report_count'] = enhanced_datasets['dataset_id'].map(report_counts).fillna(0)
    
    # Count dataflow references per dataset
    if not dataflow_refs_df.empty and 'source_dataset_id' in dataflow_refs_df.columns:
        dataflow_counts = dataflow_refs_df.groupby('source_dataset_id').size().to_dict()
        enhanced_datasets['dataflow_count'] = enhanced_datasets['dataset_id'].map(dataflow_counts).fillna(0)

# 🆕 Save Enhanced Dataset Analysis to Lakehouse
save_to_lakehouse(enhanced_datasets, "dataset_analysis", 
                 "Datasets with Reports, Tables, Relationships, Dataflows context")

In [ ]:
# ------------------------------------------------------------
# STEP 3: Usage Summary Table (Enhanced)
# ------------------------------------------------------------

summary_records = []

for _, ds in datasets_df.iterrows():
    ds_id = ds['dataset_id']
    ds_name = ds['dataset_name']
    ws_name = ds['workspace_name']

    # Reports using this dataset
    rep_refs = pbi_reports_df[pbi_reports_df['dataset_id'] == ds_id]
    paginated_refs = rep_refs[rep_refs['report_type'] == 'PaginatedReport'] if 'report_type' in rep_refs.columns else pd.DataFrame()
    normal_refs = rep_refs[rep_refs['report_type'] != 'PaginatedReport'] if 'report_type' in rep_refs.columns else rep_refs

    # Dataflows referencing this dataset (if any)
    dataflow_refs = []
    if not dataflow_refs_df.empty and 'source_dataset_id' in dataflow_refs_df.columns:
        dataflow_refs = dataflow_refs_df[dataflow_refs_df['source_dataset_id'] == ds_id]

    # Determine usage
    total_refs = len(rep_refs) + len(dataflow_refs)
    usage_status = "Unused" if total_refs == 0 else "Used"

    # Add records for all associated reports
    if not rep_refs.empty:
        for _, r in rep_refs.iterrows():
            summary_records.append({
                "Dataset_Workspace": ws_name,
                "Dataset_Name": ds_name,
                "Report_Name": r['name'],
                "Report_Type": r.get('report_type', 'PowerBIReport'),
                "Report_Workspace": r['workspace_name'],
                "Usage_Status": usage_status,
                "Total_References": total_refs
            })
    # Add records for datasets with no references
    elif total_refs == 0:
        summary_records.append({
            "Dataset_Workspace": ws_name,
            "Dataset_Name": ds_name,
            "Report_Name": None,
            "Report_Type": None,
            "Report_Workspace": None,
            "Usage_Status": usage_status,
            "Total_References": total_refs
        })

usage_summary_df = pd.DataFrame(summary_records)

display(usage_summary_df)

# 🆕 Save Usage Summary to Lakehouse
print("\n💾 Saving usage summary to lakehouse...")
save_to_lakehouse(usage_summary_df, "usage_summary", "Summary of dataset usage patterns")


In [ ]:
# ------------------------------------------------------------
# STEP 4: Enhanced Table Analysis (From Version 1)
# ------------------------------------------------------------

print("\n🔍 Enhanced Table Analysis with Usage Context...")

table_usage = []
all_dependencies = []

for _, ds in datasets_df.iterrows():
    ds_id = ds['dataset_id']
    ds_name = ds['dataset_name']
    ws_id = ds['workspace_id']
    ws_name = ds['workspace_name']
    print(f"\n🔹 Dataset: {ds_name} (Workspace: {ws_name})")    
    
    try:
        # Get model dependencies
        deps = fabric.get_model_calc_dependencies(dataset=ds_id, workspace=ws_id)
        with deps as calc_deps:
            dependencies_df = getattr(calc_deps, "dependencies_df", None)
        
        if dependencies_df is not None:
            dependencies_df['dataset_id'] = ds_id
            dependencies_df['dataset_name'] = ds_name
            dependencies_df['workspace_id'] = ws_id
            dependencies_df['workspace_name'] = ws_name
            all_dependencies.append(dependencies_df)

        # Get tables
        tables = fabric.list_tables(dataset=ds_id,workspace=ws_id)
        tables = sanitize_df_columns(tables)
        tables['workspace_id'] = ws_id
        tables['dataset_id'] = ds_id
        tables['workspace_name'] = ws_name
        tables['dataset_name'] = ds_name

        print(f" Found {len(tables)} total tables")
        
        # Get relationships
        relationships = fabric.list_relationships(dataset=ds_id, workspace=ws_id, extended=True)
        relationships['qualified_from'] = "'" + relationships['From Table'] + "'[" + relationships['From Column'] + "]"
        relationships['qualified_to'] = "'" + relationships['To Table'] + "'[" + relationships['To Column'] + "]"

        # Get measures
        measures = fabric.list_measures(dataset=ds_id, workspace=ws_id)

        # Determine used tables
        used_tables = set()
        if dependencies_df is not None:
            used_tables.update(set(dependencies_df['Referenced Table'].dropna()))
        used_tables.update(set(relationships['From Table'].dropna()))
        used_tables.update(set(relationships['To Table'].dropna()))
        used_tables.update(set(measures['Table Name'].dropna()))
        
        used_tables = {t for t in used_tables if pd.notna(t)}
        
        print(f" Found {len(used_tables)} used tables")

        # Analyze each table
        for t in set(tables['name'].dropna()):
            measures_count = len(measures[measures['Table Name'] == t])
            rel_count = len(relationships[(relationships['From Table'] == t) | (relationships['To Table'] == t)])
            dep_count = len(dependencies_df[dependencies_df['Referenced Table'] == t]) if dependencies_df is not None else 0
            status = "Unused" if t not in used_tables else "Used"
            
            table_usage.append({
                'workspace': ws_name,
                'dataset': ds_name,
                'table': t,
                'measures': measures_count,
                'relationships': rel_count,
                'dependencies': dep_count,
                "usage": status,
                'workspace_id': ws_id,
                'dataset_id': ds_id
            })
            
    except Exception as e:
        print(f"  ⚠️ Error analyzing {ds_name}: {e}")

# Convert to DataFrame and display
table_usage_df = pd.DataFrame(table_usage)
display(table_usage_df)

# 🆕 Save Table Analysis to Lakehouse
print("\n💾 Saving table analysis to lakehouse...")
save_to_lakehouse(table_usage_df, "table_analysis", 
                 "Tables with usage context from measures, relationships, and dependencies")

In [ ]:
# ------------------------------------------------------------
# STEP 5: Enhanced Column Usage Analysis (From Version 1)
# ------------------------------------------------------------

print("\n🔍 Enhanced Column Usage Analysis...")

columns_usage = []

for _, ds in datasets_df.iterrows():
    ds_id = ds['dataset_id']
    ds_name = ds['dataset_name']
    ws_id = ds['workspace_id']
    ws_name = ds['workspace_name']

    print(f"\n🔹 Dataset: {ds_name} (Workspace: {ws_name})")    

    try:
        # Get model dependencies (reuse from previous step if available)
        deps = fabric.get_model_calc_dependencies(dataset=ds_id, workspace=ws_id)
        with deps as calc_deps:
            dependencies_df = getattr(calc_deps, "dependencies_df", None)
    
        # Get relationships
        relationships = fabric.list_relationships(dataset=ds_id, workspace=ws_id, extended=True)
        relationships['qualified_from'] = "'" + relationships['From Table'] + "'[" + relationships['From Column'] + "]"
        relationships['qualified_to'] = "'" + relationships['To Table'] + "'[" + relationships['To Column'] + "]"

        # --- Get all columns in the dataset ---
        all_columns = fabric.list_columns(dataset=ds_id, workspace=ws_id, extended=True)
        all_columns = sanitize_df_columns(all_columns)
        all_columns['workspace_id'] = ws_id
        all_columns['dataset_id'] = ds_id
        all_columns['workspace_name'] = ws_name
        all_columns['dataset_name'] = ds_name
        all_columns['qualified_name'] = "'" + all_columns['table_name'] + "'[" + all_columns['column_name'] + ']'

        print(f" Found {len(all_columns)} total columns")

        # --- Filtered dependencies to only Columns and Calc Columns ---
        dep_columns_df = (
            dependencies_df[
                dependencies_df['Referenced Object Type'].isin(['Column', 'Calc Column'])
            ]
            if dependencies_df is not None else pd.DataFrame()
        )

        # --- Extract subsets by object type ---
        if not dep_columns_df.empty:
            measures_refs_df = dep_columns_df[dep_columns_df['Object Type'] == 'Measure']
            relationship_refs_df = dep_columns_df[
                dep_columns_df['Object Type'].str.contains('Relationship', case=False, na=False)
            ]
        else:
            measures_refs_df = pd.DataFrame()
            relationship_refs_df = pd.DataFrame()

        # --- Used columns (in dependencies or relationships) ---
        dep_columns = set(dep_columns_df['Referenced Full Object Name']) if not dep_columns_df.empty else set()
        rel_columns = set(relationships['qualified_from']).union(set(relationships['qualified_to']))
        used_columns = dep_columns.union(rel_columns)
        used_columns = {c for c in used_columns if pd.notna(c)}

        print(f" Found {len(used_columns)} used columns")

        # --- Determine usage per column ---
        for _, row in all_columns.iterrows():
            table_name = row['table_name']
            column_name = row['column_name']                           
            qualified_name = row['qualified_name']
            
            if pd.isna(column_name):
                continue

            dep_count = len(dep_columns_df[
                dep_columns_df['Referenced Full Object Name'] == qualified_name
            ]) if not dep_columns_df.empty else 0

            measure_c = len(measures_refs_df[measures_refs_df['Referenced Full Object Name'] == qualified_name])
            relationship_c = len(relationship_refs_df[relationship_refs_df['Referenced Full Object Name'] == qualified_name])

            # Build a referenced-by list (measures, relationships, etc.)
            referenced_by = ", ".join(
                dep_columns_df.loc[
                    dep_columns_df['Referenced Full Object Name'] == qualified_name, 'Object Name'
                ].unique().tolist()
            ) if not dep_columns_df.empty else ""

            # Determine usage
            usage_status = 'Used' if any([measure_c, relationship_c, dep_count]) else 'Unused'
            
            # Append result
            columns_usage.append({
                'workspace': ws_name,
                'dataset': ds_name,
                'table': table_name,
                'column': column_name,
                'measures': measure_c,
                'relationships': relationship_c,
                'dependencies': dep_count,
                'referenced_by': referenced_by,
                'usage': usage_status,
                'workspace_id': ws_id,
                'dataset_id': ds_id
            })
            
    except Exception as e:
        print(f"  ⚠️ Error analyzing columns for {ds_name}: {e}")

# Convert to DataFrame and display
columns_usage_df = pd.DataFrame(columns_usage)
display(columns_usage_df)

# 🆕 Save Column Usage Analysis to Lakehouse
print("\n💾 Saving column usage analysis to lakehouse...")
save_to_lakehouse(columns_usage_df, "column_usage_analysis", 
                 "Detailed column usage analysis with context from measures, relationships, and dependencies")

In [ ]:
# ------------------------------------------------------------
# STEP 6: Final Summary and Lakehouse Overview
# ------------------------------------------------------------

print("\n" + "="*80)
print("🎉 ENHANCED FABRIC WORKSPACE ANALYSIS COMPLETE")
print("="*80)

# Summary statistics
print(f"📊 Discovery Summary:")
print(f"  Workspaces: {len(workspaces_df)}")
print(f"  Datasets:   {len(datasets_df)}")
print(f"  Reports:    {len(reports_df)}")
print(f"  Dataflows:  {len(dataflows_df)}")

if table_usage:
    table_usage_summary = pd.DataFrame(table_usage)
    used_tables = len(table_usage_summary[table_usage_summary['usage'] == 'Used'])
    unused_tables = len(table_usage_summary[table_usage_summary['usage'] == 'Unused'])
    print(f"  Tables:     {len(table_usage)} (Used: {used_tables}, Unused: {unused_tables})")

if columns_usage:
    columns_usage_summary = pd.DataFrame(columns_usage)
    used_columns = len(columns_usage_summary[columns_usage_summary['usage'] == 'Used'])
    unused_columns = len(columns_usage_summary[columns_usage_summary['usage'] == 'Unused'])
    print(f"  Columns:    {len(columns_usage)} (Used: {used_columns}, Unused: {unused_columns})")

print(f"\n💾 Lakehouse Tables Created:")
print(f"  📊 workspace_analysis - Basic workspace information")
print(f"  📊 dataset_analysis - Datasets with context (Reports, Tables, Relationships, Dataflows)")
print(f"  📊 table_analysis - Tables with usage context from measures, relationships, dependencies")
print(f"  📊 column_usage_analysis - Detailed column usage analysis")
print(f"  📊 usage_summary - Summary of dataset usage patterns")

# Display final unused datasets
if not unused_datasets_df.empty:
    print("\n⚠️ UNUSED DATASETS")
    for _, row in unused_datasets_df.iterrows():
        print(f" - {row['workspace_name']} → {row['dataset_name']}")
else:
    print("\n🎉 No unused datasets found!")

print("\n" + "="*80)
print("✅ Analysis complete! Check your lakehouse for detailed results.")
print("="*80)